In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.feature as cfeature

import scipy.stats as stats

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

import shapely.geometry as sg
from shapely.geometry import Point

import sys
sys.path.append(os.path.abspath(".."))
from function import DOWN_raw

import warnings
warnings.filterwarnings('ignore')

## Export the Statistics and Extreme Quantiles for rain gauges using QC version data of Dario

In [2]:
QC_method = 'QCv1_Flag1'
# QC_method = 'QCv1_Flag_Rec'

In [3]:
yy_s, yy_e = 2002, 2023
years_num = yy_e - yy_s + 1

In [4]:
lon_min, lon_max, lat_min, lat_max, area, toll = 6.5, 19, 36.5, 48, 'ITALY', 0.002

In [5]:
dir_base = os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'QC')

In [6]:
veneto_dir = os.path.join('/','media','arturo','T9','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    REGIONS = gpd.read_file(os.path.join(veneto_dir,'Italy_regions.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

In [7]:
METADATA = pd.read_csv(os.path.join(dir_base, 'METADATA', 'METADATA_INTENSE_UPDATE.csv'))
METADATA.head(2)

,File,ISO,Region,Code,Name,Elevatn,Or_EPSG,CRS_E,CRS_N,Lat,Lon,Yr_Strt,Yer_End,INTENSE,AGENCY,Source,TimeZon
0,IT-210_PIE-001030-900_0060_QCv1.csv,IT-210,Piedmont,PIE-001030-900,BORGOFRANCODIVREA,337.0,32632,410005.0,5040731.0,45.514302,7.847798,1988-05-08 05:00:00,2024-09-09 01:00:00,DT,ARPA Piemonte,Piedmont,UTC
1,IT-210_PIE-001097-900_0060_QCv1.csv,IT-210,Piedmont,PIE-001097-900,CUMIANA,327.0,32632,373092.0,4980177.0,44.963696,7.390821,1988-01-29 03:00:00,2024-09-18 12:00:00,DT,ARPA Piemonte,Piedmont,UTC


In [8]:
dir_read = os.path.join(dir_base, 'data_1h')
regions_ISO = [d for d in os.listdir(dir_read) if os.path.isdir(os.path.join(dir_read, d))]

In [9]:
for nn in range(len(regions_ISO)):
    print(f'Directory: {regions_ISO[nn]}')
    dir_tmp = os.path.join(dir_base, 'DATA_1dy', 'statistics', QC_method, regions_ISO[nn])
    if not os.path.exists(dir_tmp):
        os.makedirs(dir_tmp)

Directory: IT-210
Directory: IT-230
Directory: IT-250
Directory: IT-321
Directory: IT-322
Directory: IT-340
Directory: IT-360
Directory: IT-420
Directory: IT-450
Directory: IT-520
Directory: IT-550
Directory: IT-570
Directory: IT-620
Directory: IT-670
Directory: IT-720
Directory: IT-750
Directory: IT-770
Directory: IT-780
Directory: IT-820
Directory: IT-880


In [10]:
for rr in range(len(regions_ISO)):#len(regions_ISO)
    region_ = regions_ISO[rr] 
    METADATA_clear = METADATA[METADATA['ISO']==region_].reset_index(inplace=False)
    
    print(f'Region: {region_} has {len(METADATA_clear)} stations')
    
    names, lats, lons, Q5, Q10, Q20, Q50, Q100 = [], [], [], [], [], [], [], []
    for nn in range(len(METADATA_clear)):#len(METADATA_clear)

        filename = f'{METADATA_clear['File'].values[nn]}'
        RAIN_pd = pd.read_csv(os.path.join(dir_base, 'DATA_1dy', 'data', region_, filename))
        RAIN_pd['Datetime'] = pd.to_datetime(RAIN_pd['Datetime'].values)
        
        ss, ee = RAIN_pd['Datetime'][0].year, RAIN_pd['Datetime'][len(RAIN_pd['Datetime'])-1].year

        RAIN_xr = xr.DataArray(
                        RAIN_pd[QC_method].values,
                        coords={'time':RAIN_pd['Datetime'].values}, 
                        dims=('time'))

        NCWy, YEARSy = DOWN_raw.fit_yearly_weibull(RAIN_xr,1)
        N = NCWy[:,0]
        C = NCWy[:,1]
        W = NCWy[:,2]
        Y = YEARSy

        mask = (N == 0)
        N[mask] = np.nan
        C[mask] = np.nan
        W[mask] = np.nan

        STATISTIC = pd.DataFrame({'Year':Y.astype(int), 'N': N, 'C': C, 'W':W})

        dir_out = os.path.join(dir_base, 'DATA_1dy', 'statistics', QC_method, region_, filename)
        STATISTIC.to_csv(dir_out, header=True, index=False)

Region: IT-210 has 38 stations
Region: IT-230 has 69 stations
Region: IT-250 has 228 stations
Region: IT-321 has 97 stations
Region: IT-322 has 57 stations
Region: IT-340 has 213 stations
Region: IT-360 has 203 stations
Region: IT-420 has 153 stations
Region: IT-450 has 439 stations
Region: IT-520 has 408 stations
Region: IT-550 has 65 stations
Region: IT-570 has 42 stations
Region: IT-620 has 213 stations
Region: IT-670 has 22 stations
Region: IT-720 has 153 stations
Region: IT-750 has 36 stations
Region: IT-770 has 37 stations
Region: IT-780 has 158 stations
Region: IT-820 has 106 stations
Region: IT-880 has 95 stations


In [11]:
STATISTIC.head(3)

,Year,N,C,W
0,2010,13.0,4.351081,0.866994
1,2011,NaN,NaN,NaN
2,2012,NaN,NaN,NaN
